## Dependencies

Install dependencies not available on Google Collab.
Collab provides numpy, pandas, sklearn, tensorflow, scipy, etc. (see requirements.txt)

In [33]:
!pip install pinard
!pip install scikeras

[autoreload of regressors failed: Traceback (most recent call last):
  File "d:\Workspace\ML\pynirsENV\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "d:\Workspace\ML\pynirsENV\lib\site-packages\IPython\extensions\autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "d:\Workspace\ML\pynirsENV\lib\site-packages\IPython\extensions\autoreload.py", line 347, in update_generic
    update(a, b)
  File "d:\Workspace\ML\pynirsENV\lib\site-packages\IPython\extensions\autoreload.py", line 317, in update_class
    update_instances(old, new)
  File "d:\Workspace\ML\pynirsENV\lib\site-packages\IPython\extensions\autoreload.py", line 278, in update_instances
    for ref in refs:
KeyboardInterrupt
]



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## Benchmark details

The results aggregate the combination of the following trainings configurations:
- estimation configuration: [regression, classification]
- datasets configurations: [Single Train, Cross validation with 5 folds and 2 repeats, Augmented Single Train]
- preprocessing configuration: [flat spectrum, savgol, haar, [small set], [big_set]]
- models: 
   - for all configuration: BACON, BACON-VG, DECON, PLS(components from 1 to 100), XGBoost, LW-PLS
   - for single train + small_set : Stack > [ BACON, BACON-VG, DECON, PLS(components from 1 to 100), XGBoost, LW-PLS,
   f_PLSRegression,f_AdaBoostRegressor,f_BaggingRegressor,f_ExtraTreesRegressor, f_GradientBoostingRegressor,f_RandomForestRegressor,
   f_ARDRegression,f_BayesianRidge,f_ElasticNet,f_ElasticNetCV,f_HuberRegressor, f_LarsCV,f_LassoCV,f_Lasso,f_LassoLars,f_LassoLarsCV,
   f_LassoLarsIC,f_LinearRegression,f_OrthogonalMatchingPursuit,f_OrthogonalMatchingPursuitCV, f_PassiveAggressiveRegressor,f_RANSACRegressor,
   f_Ridge,f_RidgeCV,f_SGDRegressor,f_TheilSenRegressor,f_GaussianProcessRegressor,f_KNeighborsRegressor, f_Pipeline,f_MLPRegressor,f_LinearSVR,
   f_NuSVR,f_SVR,f_DecisionTreeRegressor,f_ExtraTreeRegressor,f_KernelRidge,f_XGBRegressor]

We perform training in 2 steps, (1) data transformation and (2) training because the sklearn pipeline does not use test data natively.
To change with pinard update in the future.

In [3]:
### FAST GPU RESET ####
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [2]:
## Browse path and launch benchmark for every folders
%load_ext autoreload
%autoreload 2

from pathlib import Path
from preprocessings import preprocessing_list

from benchmark_loop import benchmark_dataset

import tensorflow as tf

tf.get_logger().setLevel("ERROR")
tf.keras.mixed_precision.set_global_policy("mixed_float16")

rootdir = Path('data/regression')
folder_list = [f for f in rootdir.glob('**/*') if f.is_dir()]

SEED = ord('D') + 31373

# tf.keras.utils.set_random_seed(SEED)
# tf.config.experimental.enable_op_determinism()


import preprocessings
import regressors
import pinard.preprocessing as pp
from pinard import augmentation, model_selection
from sklearn.cross_decomposition import PLSRegression
from xgboost import XGBRegressor
import sys
import os.path

def str_to_class(classname):
    return getattr(sys.modules['pinard.preprocessing'], classname)

# print(str_to_class('SavitzkyGolay'))




def get_dataset_list(path):
    datasets = []
    for r, d, _ in os.walk(path):
        for folder in d:
            path = os.path.join(r, folder)
            if os.path.isdir(path):
                # if len(datasets) < 3:
                datasets.append(str(path))
    return datasets

split_configs = [
    None,
    # {'test_size':None, 'method':"random", 'random_state':SEED},
    # {'test_size':None, 'method':"stratified", 'random_state':SEED, 'n_bins':5},
    # {'test_size':0.25, 'method':"spxy", 'random_state':SEED, 'metric':"euclidean", 'pca_components':250},
]

augmentations = [
    None,
    # [(6, augmentation.Rotate_Translate())],
    # [(3, augmentation.Rotate_Translate()),(2, augmentation.Random_X_Operation()),(1, augmentation.Random_Spline_Addition()),],
    # [(3, augmentation.Rotate_Translate()),(2, augmentation.Random_X_Operation()),(2, augmentation.Random_Spline_Addition()),]
]

preprocessings_list = [
    # None,
    # preprocessings.id_preprocessing(),
    [("id", pp.IdentityTransformer()), ('haar', pp.Haar()), ('savgol', pp.SavitzkyGolay())],
    # preprocessings.decon_set(),
    # preprocessings.bacon_set(),
    # preprocessings.small_set(),
    # preprocessings.transf_set(),
    # preprocessings.optimal_set_2D(),
    # preprocessings.fat_set(),
]



cv_configs = [
    None,
    # {'n_splits':5, 'n_repeats':4},
    # {'n_splits':4, 'n_repeats':2},
    # {'n_splits':3, 'n_repeats':1},
]

# import os
folder = "data/regression"
folders = get_dataset_list(folder)
print(folders)
# folders = ["data/regression/Cassava_TBC_3556_Davrieux_RMSE1.02"]

len_cv_configs = 0
for c in cv_configs:
    if c == None:
        len_cv_configs += 1
    else:
        len_cv_configs += (c['n_splits'] * c['n_repeats'])

models = [
    # (regressors.ML_Regressor(XGBRegressor), {"n_estimators":200, "max_depth":50, "seed":SEED}),
    # (regressors.ML_Regressor(PLSRegression), {"n_components":50}),
    # (regressors.Transformer_NIRS(), {'batch_size':500, 'epoch':10000, 'verbose':0, 'patience':1000, 'optimizer':'adam', 'loss':'mse'}),
    # (regressors.Decon_SepPo(), {'batch_size':50, 'epoch':10000, 'verbose':0, 'patience':1000, 'optimizer':'adam', 'loss':'mse'}),
    # (regressors.FFT_Conv(), {'batch_size':500, 'epoch':20000, 'verbose':0, 'patience':1000, 'optimizer':'adam', 'loss':'mse'}),
    # (regressors.Decon(), {'batch_size':100, 'epoch':20000, 'verbose':0, 'patience':400, 'optimizer':'adam', 'loss':'mse'}),
    (regressors.Decon_Sep(), {'batch_size':100, 'epoch':20000, 'verbose':0, 'patience':200, 'optimizer':'adam', 'loss':'mse'}),
    # (regressors.ResNetV2(), {'batch_size':200, 'epoch':20000, 'verbose':0, 'patience':300, 'optimizer':'adam', 'loss':'mse'}),
    # (regressors.MLP(), {'batch_size':1000, 'epoch':20000, 'verbose':0, 'patience':2000, 'optimizer':'adam', 'loss':'mse'}),
    # (regressors.CONV_LSTM(), {'batch_size':1000, 'epoch':20000, 'verbose':0, 'patience':2000, 'optimizer':'adam', 'loss':'mse'}),
    # (regressors.XCeption1D(), {'batch_size':500, 'epoch':10000, 'verbose':0, 'patience':1200, 'optimizer':'adam', 'loss':'mse'}),
    # (regressors.Transformer(), {'batch_size':10, 'epoch':300, 'verbose':0, 'patience':30, 'optimizer':'Adam', 'loss':'mse'}),
]

benchmark_size = len(folders) * len(split_configs) * len_cv_configs * len(augmentations) * len(preprocessings_list) * len(models)
print("Benchmarking", benchmark_size, "runs.")


benchmark_dataset(folders, split_configs, cv_configs, augmentations, preprocessings_list, models, SEED) #, resampling='resample', resample_size=2048) #bins=5)
# benchmark_dataset(folders, split_configs, cv_configs, augmentations, preprocessings_list, models, SEED, resampling='crop', resample_size=2150)


# for folder in folder_list:
    # # print(ord(str(folder)[17]), ord('A'), ord('M'))
    # if ord(str(folder)[16]) < ord("L") or ord(str(folder)[16]) > ord("M"):
    #     continue
    # benchmark_dataset(folder, SEED, preprocessing_list(), 20, augment=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['data/regression\\ALPINE_C_424_Murguzur_RMSE1.16', 'data/regression\\Cassava_TBC_3393_Sanchez_RMSE1.12', 'data/regression\\LUCAS_SOCgrassland_4096_Nocita_RMSE7.2', 'data/regression\\Meat_Fat_215_Borggaard_RMSE0.65']
Benchmarking 4 runs.
========== ALPINE_C_424_MURGUZUR_RMSE1.16 ( resample 2048 ) 

d:\Workspace\ML\DECON\data.py:81: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,2

Missing X: [0 0 0 ... 0 0 0]
========== (361, 2048) (361, 1) (63, 2048) (63, 1)
ResNetV2--NoSpl-NoCV-Fold_1(1)-NoAug-PP_22_23906-31441-23-02-08_16-21-21 (361, 2048) (361, 1) (63, 2048, 22) (63, 1)
--- Trainable: 1432061 - untrainable: 12908 > 1444969


********** (63, 2048) (63, 1) (63, 1)
Epoch: 0 > RMSE: 75.098  ( -1 | None ) - R²: -1152.1123005585287  val_loss 10.078125
********** (63, 2048) (63, 1) (63, 1)625
Epoch: 1 > RMSE: 48.117783  ( -1 | None ) - R²: -472.3976925462145  val_loss 4.140625
********** (63, 2048) (63, 1) (63, 1)90625
Epoch: 2 > RMSE: 44.809498  ( -1 | None ) - R²: -409.5396022251484  val_loss 3.587890625
********** (63, 2048) (63, 1)   2.294921875(63, 1)
Epoch: 3 > RMSE: 35.829994  ( -1 | None ) - R²: -261.4873422058601  val_loss 2.294921875
********** (63, 2048) (63, 1) (63, 1)9453125
Epoch: 4 > RMSE: 14.734008  ( -1 | None ) - R²: -43.387093704555284  val_loss 0.387939453125
********** (63, 2048) (63, 1) (63, 1)833984375
Epoch: 5 > RMSE: 13.58573  ( -1 | None ) - R²: -36.738170051368584  val_loss 0.329833984375
********** (63, 2048) (63, 1) (63, 1)82421875
Epoch: 7 > RMSE: 13.370371  ( -1 | None ) - R²: -35.55121184307552  val_loss 0.31982421875
********** (63, 2048) (63, 1) (63, 1)6328125
Epoch: 8 > RMSE: 11

d:\Workspace\ML\DECON\data.py:81: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,2

Missing X: [0 0 0 ... 0 0 0]
========== (1823, 2048) (1823, 1) (1270, 2048) (1270, 1)
ResNetV2--NoSpl-NoCV-Fold_1(1)-NoAug-PP_22_23906-31441-23-02-08_16-25-37 (1823, 2048) (1823, 1) (1270, 2048, 22) (1270, 1)
--- Trainable: 1432061 - untrainable: 12908 > 1444969


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:GPU:0 to /job:localhost/replica:0/task:0/device:CPU:0 in order to run TensorDataset: Dst tensor is not initialized. [Op:TensorDataset]

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
import numpy as np
y_train = np.array([0,1,2,2.5,3,3.5,6,8,20]).reshape(-1,1)
bins = 4
discretizer = KBinsDiscretizer(n_bins=bins, encode='onehot-dense', strategy='uniform')
discretizer.fit(y_train)
tt = discretizer.transform(y_train)
print(tt)


[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]


In [ ]:
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_validate, cross_val_predict
import numpy as np

def plot_regression_results(ax, y_true, y_pred, title, scores, elapsed_time):
    """Scatter plot of the predicted vs true targets."""
    ax.plot(
        [y_true.min(), y_true.max()], [y_true.min(), y_true.max()], "--r", linewidth=2
    )
    ax.scatter(y_true, y_pred, alpha=0.2)

    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()
    ax.spines["left"].set_position(("outward", 10))
    ax.spines["bottom"].set_position(("outward", 10))
    ax.set_xlim([y_true.min(), y_true.max()])
    ax.set_ylim([y_true.min(), y_true.max()])
    ax.set_xlabel("Measured")
    ax.set_ylabel("Predicted")
    extra = plt.Rectangle(
        (0, 0), 0, 0, fc="w", fill=False, edgecolor="none", linewidth=0
    )
    ax.legend([extra], [scores], loc="upper left")
    title = title + "\n Evaluation in {:.2f} seconds".format(elapsed_time)
    ax.set_title(title)

def plot_data(d, filepath):
    plt.scatter(d[:,0], d[:,1])
    plt.xlabel('test')
    plt.ylabel('predict')
    plt.savefig(filepath + '.png')
    plt.close()

import json
from numpy import genfromtxt

path = 'results'
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.csv'):
            filepath = os.path.join(root, file)
            df = pd.read_csv(filepath)
            my_data = genfromtxt(filepath, delimiter=';')
            # print(my_data)
            plot_data(my_data, filepath.replace('csv','png'))
        # if file.endswith('.json'):
        #     print(file)
        #     dataset = file.replace('.json','')
        #     f = open(os.path.join(root, file))
        #     data = json.load(f)
        #     for key in data:
        #         print(key)

# returns JSON object as 
# a dictionary
            # filepath = os.path.join(root, file)
            # df = pd.read_csv(filepath)
            # y_res = df.iloc[:,0]
            # y_pred = df.iloc[:,1]
            # fig, axs = plt.subplots(1,1, figsize=(10,10))
            # axs = np.ravel(axs)

            # plot_regression_results(
            #     ax,
            #     y,
            #     y_pred,
            #     name,
            #     (r"$R^2={:.2f} \pm {:.2f}$" + "\n" + r"$MAE={:.2f} \pm {:.2f}$").format(
            #         np.mean(score["test_r2"]),
            #         np.std(score["test_r2"]),
            #         -np.mean(score["test_neg_mean_absolute_error"]),
            #         np.std(score["test_neg_mean_absolute_error"]),
            #     ),
            #     elapsed_time,
            # )

    



# plt.suptitle("Single predictors versus stacked predictors")
# plt.tight_layout()
# plt.subplots_adjust(top=0.9)
# plt.show()

In [ ]:
from pinard.utils import load_csv
from benchmark_loop import transform_test_data
import preprocessings
import numpy as np

dumb_set = preprocessings.dumb_and_dumber_set()
Xfile = "data/regression/ALPINE_Calpine_424_Murguzur_RMSE1.36/Xcal.csv.gz"
yfile = "data/regression/ALPINE_Calpine_424_Murguzur_RMSE1.36/Ycal.csv.gz"
X_train, y_train = load_csv(Xfile, yfile, x_hdr=0, y_hdr=0, sep=";")
X_train, y_train, X_test, y_test = X_train[0:100], y_train[0:100], X_train[0:100], y_train[0:100]
X_test_pp, y_test_pp, transformer_pipeline, y_scaler = transform_test_data(dumb_set, X_train, y_train, X_test, y_test, type="augmentation")

print(X_test_pp.shape)
sample = X_test_pp[0]
print(len(dumb_set))
ok = []
for i in range(len(dumb_set)-1, -1, -1):
    found = False
    for j in range(i-1, -1, -1):
        if np.allclose(sample[i], sample[j], rtol=10e-3, atol=10e-3):
            found = True
            break
    if not found:
        ok.append(dumb_set[i][0])

print(len(ok))
print(ok)

In [ ]:
import os
import csv

def count_columns(file):
    with open(file, 'r') as f:
        reader = csv.reader(f, delimiter=";")
        header = next(reader)
        header = next(reader)
        return len(header), header

def walk_directory(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.csv'):
                path = os.path.join(root, file)
                columns, header = count_columns(path)
                if columns > 2100 and columns < 4000:
                    print(header[0:5])
                    print(f"File: {path}, Columns: {columns}")

walk_directory('rawdata/regression') # Replace '.' with the directory path you want to traverse


In [4]:
import csv
import json
import os

def json_to_csv(json_folder, csv_file):
    csv_rows = []
    for file in os.listdir(json_folder):
        if file.endswith(".json"):
            file_path = os.path.join(json_folder, file)
            with open(file_path, "r") as f:
                json_data = json.load(f)
            for key, obj in json_data.items():
                csv_row = {}
                for key, value in obj.items():
                    csv_row[key] = value
                csv_rows.append(csv_row)

    fieldnames = []
    for row in csv_rows:
        for key in row.keys():
            if key not in fieldnames:
                fieldnames.append(key)

    with open(csv_file, "w") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for row in csv_rows:
            writer.writerow(row)

json_folder = "results"
csv_file = "result.csv"

json_to_csv(json_folder, csv_file)

results/ALPINE_Calpine_424_Murguzur_RMSE1.36_results.json
{'model': 'Decon_Sep--NoSpl-CV_4_1-NoAug-PP_110_-2271-31441-23-02-11_16-19-26', 'dataset': 'ALPINE_Calpine_424_Murguzur_RMSE1.36', 'seed': '31441', 'RMSE': '2.737399', 'MAPE': '0.037886858', 'R2': '0.06202538829456827', 'MAE': '1.6973672', 'MSE': '7.4933543', 'MedAE': '1.0064926', 'EVS': '0.06645399332046509', 'run': '2023-02-11  16:28:05', 'path': 'data/regression/ALPINE_Calpine_424_Murguzur_RMSE1.36'}
{'model': 'Decon_Sep--NoSpl-CV_4_1-NoAug-PP_110_-7444-31441-23-02-11_18-33-21', 'dataset': 'ALPINE_Calpine_424_Murguzur_RMSE1.36', 'seed': '31441', 'RMSE': '2.7761836', 'MAPE': '0.04098059', 'R2': '0.03525799268526908', 'MAE': '1.8505303', 'MSE': '7.7071953', 'MedAE': '1.29912', 'EVS': '0.06998336315155029', 'run': '2023-02-11  18:41:34', 'path': 'data/regression/ALPINE_Calpine_424_Murguzur_RMSE1.36'}
{'model': 'Decon_Sep--NoSpl-NoCV-Fold_1(1)-NoAug-PP_110_-7444-31441-23-02-11_18-31-47', 'dataset': 'ALPINE_Calpine_424_Murguzur_RM